In [ ]:
import pandas as pd
import numpy as np
import random
import math 
import sys
sys.setrecursionlimit(10000)

In [ ]:
def df_to_array(excel_data):
  data = pd.read_excel(excel_data)
  tasks = list(range(0, len(data)))
  data.pop('Zadanie')
  times = data
  times = np.array(times)
  return times, tasks

def generateRandomSolution(indices):
  indices = indices.copy()
  randomSolution = []
  for i in range(0, len(indices)):
    randomIndex = random.randint(0, len(indices) - 1)
    randomSolution.append(indices[randomIndex])
    indices.pop(randomIndex)
  return randomSolution

def generateCompletedTime(times, tasks):
  completedTime = times.copy()
  machinecCount = len(times[0])
  # pierwszy wiersz
  # j - index kolumny, i - index wiersza
  completedTime[0][0] = times[tasks[0]][0]
  i = 0
  for j in range(1, machinecCount):
    completedTime[i][j] = completedTime[i][j-1] + times[tasks[i]][j]
  # pierwsza kolumna
  # j - index kolumny, i - index wiersza
  j = 0
  for i in range(1, len(tasks)):
    completedTime[i][j] = completedTime[i-1][j] + times[tasks[i]][j]
  # środek
  # j - index kolumny, i - index wiersza
  for i in range(1, len(tasks)):
    for j in range(1, machinecCount):
      maxi = max(completedTime[i-1][j], completedTime[i][j-1])
      completedTime[i][j] = maxi + times[tasks[i]][j]
  return completedTime[len(tasks)-1][machinecCount-1]

def changeIndices(indices):
  return list(map(lambda x: x+1, indices))

def createCombinations(indices):
  combinations = []
  for i in range(0, len(indices)):
    for j in range(i+1, (len(indices))):
      combinations.append([indices[i], indices[j]])
  return combinations

def divisionNeighbour(array, low, high):
  pivot = array[high][1]
  i = low - 1
  for j in range(low, high):
      if array[j][1] <= pivot:
          i = i + 1
          (array[i][1], array[j][1]) = (array[j][1], array[i][1])
          (array[i][0], array[j][0]) = (array[j][0], array[i][0])
          (array[i][2], array[j][2]) = (array[j][2], array[i][2])
  (array[i + 1][1], array[high][1]) = (array[high][1], array[i + 1][1])
  (array[i + 1][0], array[high][0]) = (array[high][0], array[i + 1][0])
  (array[i + 1][2], array[high][2]) = (array[high][2], array[i + 1][2])
  return i + 1
  
def quickSortNeighboursByF(array, low, high):
  if low < high:
      pi = divisionNeighbour(array, low, high)
      quickSortNeighboursByF(array, low, pi - 1)
      quickSortNeighboursByF(array, pi + 1, high)

def createNeighbourhoodsSwap(parent, combinations, data):
  neighbours = []
  for i in range(len(combinations)):
    solution = []
    for j in range(len(parent)):
      if parent[j] == combinations[i][0]:
        solution.append(combinations[i][1])
      elif parent[j] == combinations[i][1]:
        solution.append(combinations[i][0])
      else:
        solution.append(parent[j])
    neighbours.append([solution, generateCompletedTime(data, solution), combinations[i]])
  quickSortNeighboursByF(neighbours, 0, len(neighbours) - 1)
  # sorted(neighbours, key=lambda d: d[1]) 
  return neighbours

def createNeighbourhoodsInvert(parent, combinations, data):
  neighbours = []
  for k in range(len(combinations)):
    solution = []
    index1 = combinations[k][0]
    index2 = combinations[k][1]
    Xlen = len(parent)
    if index1 > index2:
      helper = index1
      index1 = index2
      index2 = helper
    i = 0
    j = 0
    while i < index1:
      solution.append(parent[i])
      i = i + 1
    i = index2
    while i >= index1:
      solution.append(parent[i])
      i = i - 1
    i = index2 + 1
    while i <= Xlen - 1:
      solution.append(parent[i])
      i = i + 1
    neighbours.append([solution, generateCompletedTime(data, solution), combinations[k]])
  quickSortNeighboursByF(neighbours, 0, len(neighbours) - 1)
  return neighbours

def changeIndices(indices):
  return list(map(lambda x: x+1, indices))
  
def createSolutionInvert(X):
  arr = []
  Xlen = len(X)
  index1 = random.randint(0, Xlen - 1)
  index2 = random.randint(0, Xlen - 1)
  while index1 == index2:
    index2 = random.randint(0, Xlen - 1)
  if index1 > index2:
    helper = index1
    index1 = index2
    index2 = helper
  i = 0
  j = 0
  while i < index1:
    arr.append(X[i])
    i = i + 1
  i = index2
  while i >= index1:
    arr.append(X[i])
    i = i - 1
  i = index2 + 1
  while i <= Xlen - 1:
    arr.append(X[i])
    i = i + 1
  return arr

def createSolutionSwap(X):
  arr = []
  Xlen = len(X)
  index1 = random.randint(0, Xlen - 1)
  index2 = random.randint(0, Xlen - 1)
  for j in range(Xlen):
      if X[j] == index1:
        arr.append(index2)
      elif X[j] == index2:
        arr.append(index1)
      else:
        arr.append(X[j])
  return arr

def changeIndicesInCombination(combination):
  combination[0], combination[1] = combination[1], combination[0]
  return combination

def reduceTemperatureGeometric(temperature, alfa): 
  return temperature * alfa

def reduceTemperatureSlow(temperature, alfa): 
  return temperature/(1 + alfa * temperature)

def TabuSearch(data_name, createNeighbourhoodOfType, term, iterations):
  iterationsWithoutImprovementLimit = iterations / 2 
  iterationsWithoutImprovement = 0
  times, tasks = df_to_array(data_name)
  X0 = generateRandomSolution(tasks)
  F0 = generateCompletedTime(times, X0)
  Xbest = X0
  Fbest = F0
  Xcurrent = X0
  Fcurrent = F0
  tabuList = []
  for i in range(iterations):
    if iterationsWithoutImprovement >= iterationsWithoutImprovementLimit:
      print("exceeded limit")
      break
    combinations = createCombinations(Xcurrent)
    neighbours = createNeighbourhoodOfType(Xcurrent, combinations, times)
    neighbour = neighbours[0]
    Xcurrent = neighbour[0]
    Fcurrent = neighbour[1]
    currentComb = neighbour[2]
    if currentComb[0] > currentComb[1]:
      changeIndices(currentComb)
    j = 0
    while currentComb in tabuList:
      j = j + 1
      currentComb = neighbours[j][2]
      if currentComb[0] > currentComb[1]:
        changeIndices(currentComb)
    neighbour = neighbours[j]
    Xcurrent = neighbour[0]
    Fcurrent = neighbour[1]
    if Fcurrent < Fbest:
      Fbest = Fcurrent
      Xbest = Xcurrent
      iterationsWithoutImprovement = 0
    else:
      iterationsWithoutImprovement += 1
    tabuList.append(currentComb)
    if(i > term): 
      tabuList.pop(0)
  Xbest = changeIndices(Xbest)
  return Fbest, Xbest

def SimulatedAnnealing(data_name, createNeighbourhoodOfType, initialT, alpha, iterations, reduceTemperature):
  iterationsWithoutImprovementLimit = iterations / 3
  iterationsWithoutImprovement = 0
  times, tasks = df_to_array(data_name)
  X0 = generateRandomSolution(tasks)
  F0 = generateCompletedTime(times, X0)
  Xbest = X0
  Fbest = F0
  Xcurrent = X0
  Fcurrent = F0
  currentT = float(initialT)
  for i in range(iterations):
    if iterationsWithoutImprovement >= iterationsWithoutImprovementLimit:
      print("exceeded limit")
      break
    Xpropose = createSolutionInvert(Xcurrent)
    Fpropose = generateCompletedTime(times, Xpropose)
    error =  float(Fcurrent - Fpropose)
    if error >= 0:
      Fcurrent = Fpropose
      Xcurrent = Xpropose
    else:
      pAccept = math.exp(error/currentT)
      if random.uniform(0, 1) < pAccept:
        Fcurrent = Fpropose
        Xcurrent = Xpropose
    if Fcurrent < Fbest:
      Fbest = Fcurrent
      Xbest = Xcurrent
      iterationsWithoutImprovement = 0
    else:
      iterationsWithoutImprovement += 1
    currentT = reduceTemperature(currentT, alpha)
  Xbest = changeIndices(Xbest)
  return Fbest, Xbest